Nous allons détailler les fonctions dans ce notebook.

# IMPORTS , FONCTIONS ET URL

In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options # en vue de faire tourner le browser en headless
from pathlib import Path
import os
# from webdriver_manager.chrome import ChromeDriverManager
# import chromedriver_binary
import re
import csv
import time
regex_expression = "</span> #.* in Kindle Store " # utile pour regex
#DRIVER_PATH_chromedriver = str(Path('../chromedriver').resolve()) ancienne ligne qui fonctionnait

from amazoncaptcha import AmazonCaptcha #librairi pour passer l'image capthca d'amazon
from selenium.webdriver.common.by import By




/home/pi/.local/lib/python3.9/site-packages/pandas/core/computation/expressions.py:20: UserWarning: Pandas requires version '2.7.3' or newer of 'numexpr' (version '2.7.2' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
/home/pi/.local/lib/python3.9/site-packages/pandas/core/arrays/masked.py:59: UserWarning: Pandas requires version '1.3.2' or newer of 'bottleneck' (version '1.2.1' currently installed).
  from pandas.core import (


In [2]:
urlcaptcha = "https://www.amazon.com/errors/validateCaptcha"

#attention sur PC DELL UBUNTU le Wagon, bien penser à lancer X launch , et à désactiver VPN !!

In [3]:
configuration = { "wagon":"/usr/bin/chromedriver/chromedriver" , 
                 "raspberry":"/usr/bin/chromedriver" ,
                "anaconda-PS-stock":""}

In [5]:
environnement = input('Entrez la configuration matérielle')

Entrez la configuration matérielleraspberry


In [6]:
chemin_access = configuration[environnement]

In [7]:
chemin_access

'/usr/bin/chromedriver'

In [44]:
DRIVER_PATH_chromedriver = str(Path(chemin_access).resolve()) # installation chrome driver sur usr/bin


chrome_options = Options()
chrome_options.add_argument("--headless")

browser = webdriver.Chrome(executable_path=DRIVER_PATH_chromedriver, options=chrome_options)

# browser = webdriver.Chrome(executable_path=DRIVER_PATH_chromedriver)

browser.get("http://www.amazon.com")

#browser.get("http://www.amazon.com") #ici chrome est norma
#browser.get("http://www.amazon.com" , options = chome_options) # avec cette option le chrome est headless

/tmp/ipykernel_2374/2879895388.py:7: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  browser = webdriver.Chrome(executable_path=DRIVER_PATH_chromedriver, options=chrome_options)


In [45]:
browser = webdriver.Chrome(executable_path=DRIVER_PATH_chromedriver)
browser.get("https://www.amazon.com/errors/validateCaptcha")

/tmp/ipykernel_2374/2394049935.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  browser = webdriver.Chrome(executable_path=DRIVER_PATH_chromedriver)


In [11]:
link = browser.find_element(By.XPATH,"//div[@class = 'a-row a-text-center']//img").get_attribute('src')
captcha = AmazonCaptcha.fromlink(link)
captcha_value = AmazonCaptcha.solve(captcha)
input_field = browser.find_element(By.ID, "captchacharacters").send_keys(captcha_value)
button = browser.find_element(By.CLASS_NAME,"a-button-text")
button.click()

In [12]:
def passer_le_captcha():
    link = browser.find_element(By.XPATH,"//div[@class = 'a-row a-text-center']//img").get_attribute('src')
    captcha = AmazonCaptcha.fromlink(link)
    captcha_value = AmazonCaptcha.solve(captcha)
    input_field = browser.find_element(By.ID, "captchacharacters").send_keys(captcha_value)
    button = browser.find_element(By.CLASS_NAME,"a-button-text")
    button.click()

In [25]:
browser = webdriver.Chrome(executable_path=DRIVER_PATH_chromedriver)
browser.get("https://www.amazon.com/errors/validateCaptcha")
passer_le_captcha()

/tmp/ipykernel_2374/2005360394.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  browser = webdriver.Chrome(executable_path=DRIVER_PATH_chromedriver)


In [13]:
captcha_value

'ULAXLN'

In [36]:
liste_categorie = pd.read_csv('liste_categorie.txt' , sep ='\t', index_col = "Numero")

In [31]:
liste_categorie['url'][2]

'https://www.amazon.com/s?bbn=157632011&rh=n%3A133140011%2Cn%3A154606011%2Cn%3A154607011%2Cn%3A157630011%2Cn%3A157632011%2Cn%3A157635011&dc&qid=1689403032&rnid=157632011&ref=lp_157632011_nr_n_1'

In [32]:
liste_categorie_avec_Captcha = liste_categorie

In [37]:
liste_categorie_avec_Captcha['url'].iloc[1] = "urlcaptcha"

In [38]:
liste_categorie_avec_Captcha

,url,cat1,cat2,cat3,cat4,cat5,cat6
Numero,,,,,,,
0,https://www.amazon.com/s?bbn=157630011&rh=n%3A...,Arts & Photography,Architecture,"Architects, A-Z",NaN,NaN,NaN
1,urlcaptcha,Arts & Photography,Architecture,Building Types & Styles,Religious Buildings,NaN,NaN
2,https://www.amazon.com/s?bbn=157632011&rh=n%3A...,Arts & Photography,Architecture,Building Types & Styles,Specific Styles,NaN,NaN
3,https://www.amazon.com/s?bbn=157630011&rh=n%3A...,Arts & Photography,Architecture,Criticism,NaN,NaN,NaN
4,https://www.amazon.com/s?bbn=157630011&rh=n%3A...,Arts & Photography,Architecture,Drawing & Modelling,NaN,NaN,NaN
...,...,...,...,...,...,...,...
5133,https://www.amazon.com/s?bbn=160000011&rh=n%3A...,Travel,United States,States,Washington,NaN,NaN
5134,https://www.amazon.com/s?bbn=160000011&rh=n%3A...,Travel,United States,States,West Virginia,NaN,NaN
5135,https://www.amazon.com/s?bbn=160000011&rh=n%3A...,Travel,United States,States,Wisconsin,NaN,NaN


In [39]:
liste_categorie

,url,cat1,cat2,cat3,cat4,cat5,cat6
Numero,,,,,,,
0,https://www.amazon.com/s?bbn=157630011&rh=n%3A...,Arts & Photography,Architecture,"Architects, A-Z",NaN,NaN,NaN
1,https://www.amazon.com/s?bbn=157632011&rh=n%3A...,Arts & Photography,Architecture,Building Types & Styles,Religious Buildings,NaN,NaN
2,https://www.amazon.com/s?bbn=157632011&rh=n%3A...,Arts & Photography,Architecture,Building Types & Styles,Specific Styles,NaN,NaN
3,https://www.amazon.com/s?bbn=157630011&rh=n%3A...,Arts & Photography,Architecture,Criticism,NaN,NaN,NaN
4,https://www.amazon.com/s?bbn=157630011&rh=n%3A...,Arts & Photography,Architecture,Drawing & Modelling,NaN,NaN,NaN
...,...,...,...,...,...,...,...
5133,https://www.amazon.com/s?bbn=160000011&rh=n%3A...,Travel,United States,States,Washington,NaN,NaN
5134,https://www.amazon.com/s?bbn=160000011&rh=n%3A...,Travel,United States,States,West Virginia,NaN,NaN
5135,https://www.amazon.com/s?bbn=160000011&rh=n%3A...,Travel,United States,States,Wisconsin,NaN,NaN


In [20]:
liste_categorie.groupby(["cat1"]).count()

,url,cat2,cat3,cat4,cat5,cat6
cat1,,,,,,
Arts & Photography,198,198,195,119,31,0
Biographies & Memoirs,104,104,98,45,9,0
Business & Money,213,213,211,55,2,0
Children s eBooks,363,363,363,185,17,0
"Comics, Manga , Graphic Novel",1,0,0,0,0,0
Computers & Technology,75,75,66,3,0,0
"Cookbooks, Food & Wine",114,114,106,40,0,0
"Crafts, Hobbies & Home Kindle",191,191,190,41,0,0
Education & Teaching Kindle,81,81,81,54,4,0


# DECODER LES URL DES  DES CATEGORIES ET SOUS CATEGORIES

In [21]:
liste_categorie['url'][0]

'https://www.amazon.com/s?bbn=157630011&rh=n%3A133140011%2Cn%3A154606011%2Cn%3A154607011%2Cn%3A157630011%2Cn%3A157631011&dc&qid=1689403029&rnid=157630011&ref=lp_157630011_nr_n_0'

In [68]:
liste_categorie['url'][0].split('&dc')[0].split('%')[-1]

'3A157631011'

In [70]:
url_problem = "https://www.amazon.com/s?i=digital-text&rh=n%3A154630011&fs=true&page=1"
browser = webdriver.Chrome(executable_path=DRIVER_PATH_chromedriver)
browser.get(url_problem)

/tmp/ipykernel_2374/2200296252.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  browser = webdriver.Chrome(executable_path=DRIVER_PATH_chromedriver)


In [71]:
url_sans_problem = "https://www.amazon.com/s?i=digital-text&rh=n%3A157631011&fs=true&page=4"

In [40]:
def trouver_node_url(url):
    """fonction qui retourne un lien simplifié et que l'on peu incrémenter d'une catégorie"""
    node = url.split('&dc')[0].split('%')[-1]
    lien = 'https://www.amazon.com/s?i=digital-text&rh=n%' + node +'&fs=true&page=1'
    #browser.get(lien)
    return(lien)

def trouver_node_url_a_incrementer(url):
    """fonction qui retourne un url que l'on peu incrémenter simplement il n'y a plus qu'a adjoindre i et l'incrémenter à la fin"""
    node = url.split('&dc')[0].split('%')[-1]
    lien = 'https://www.amazon.com/s?i=digital-text&rh=n%' + node +'&fs=true&page='
    #browser.get(lien)
    return(lien)

def y_a_t_il_des_livres_dans_page(url): #fonction qui détermine s'il y a des livres dans la page
    """ retourne True s'il y a des livres dans la page et False s'il n'y en a pas"""
    browser.get(url) #navigue sur une page avec tous les liens de la liste
    html_page_listing = browser.page_source # permet de prendre le code html de la page 
    soup = BeautifulSoup(html_page_listing , 'lxml') # créer la soupe avec le parser
    alerte = soup.find_all("p", class_="a-last")   # pour faire le prog robuste je copie cette partie
    while  str(alerte) == """[<p class="a-last">Sorry, we just need to make sure you\'re not a robot. For best results, please make sure your browser is accepting cookies.</p>]""" :
        passer_le_captcha()
        browser.get(url)
        html_page_listing = browser.page_source # permet de prendre le code html de la page
        soup = BeautifulSoup(html_page_listing , 'lxml')
        alerte = soup.find_all("p", class_="a-last") # à la fin de while
    livres = soup.find_all('div' ,{ 'data-asin':True, 'data-component-type' : 's-search-result'} ) # trouve tous les livres de la pages
    if livres!=[]:
        return True
    else : 
        return False
    
    
def trouver_les_livres_dans_page(url_page_principale):
    """fonction qui retourne une liste de lien web des livres présents dans la page"""
    browser.get(url_page_principale)
    html_page_listing = browser.page_source # permet de prendre le code html de la page 
    soup = BeautifulSoup(html_page_listing , 'lxml') # créer la soupe avec le parser
    alerte = soup.find_all("p", class_="a-last")   # pour faire le prog robuste je copie cette partie
    while  str(alerte) == """[<p class="a-last">Sorry, we just need to make sure you\'re not a robot. For best results, please make sure your browser is accepting cookies.</p>]""" :
        passer_le_captcha()
        browser.get(url_page_principale)
        html_page_listing = browser.page_source # permet de prendre le code html de la page
        soup = BeautifulSoup(html_page_listing , 'lxml')
        alerte = soup.find_all("p", class_="a-last") # à la fin de while  
    livres = soup.find_all('a',class_ = 'a-link-normal s-underline-text s-underline-link-text s-link-style a-text-normal')
    liens_livres=['https://www.amazon.com'+livre.get('href') for livre in livres]  
    #on utilise liste comprehension pour prendre l'élement href (lien hypertexte) 
    # et ajouter www.amazon.com/ pour avoir un url exploitable
    return(liens_livres)

def compter_nb_livre_dans_page(url_page_principale):
    """fonction qui retourne une liste de lien web des livres présents dans la page"""
    browser.get(url_page_principale)
    html_page_listing = browser.page_source # permet de prendre le code html de la page 
    soup = BeautifulSoup(html_page_listing , 'lxml') # créer la soupe avec le parser
    alerte = soup.find_all("p", class_="a-last")   # pour faire le prog robuste je copie cette partie
    while  str(alerte) == """[<p class="a-last">Sorry, we just need to make sure you\'re not a robot. For best results, please make sure your browser is accepting cookies.</p>]""" :
        passer_le_captcha()
        browser.get(url_page_principale)
        html_page_listing = browser.page_source # permet de prendre le code html de la page
        soup = BeautifulSoup(html_page_listing , 'lxml')
        alerte = soup.find_all("p", class_="a-last") # à la fin de while 
    livres = soup.find_all('a',class_ = 'a-link-normal s-underline-text s-underline-link-text s-link-style a-text-normal')
    liens_livres=['https://www.amazon.com'+livre.get('href') for livre in livres]  
    #on utilise liste comprehension pour prendre l'élement href (lien hypertexte) 
    # et ajouter www.amazon.com/ pour avoir un url exploitable
    return(len(liens_livres))

def rang_livre(url_d_un_livre):
    """ fonction qui retour le rang des vente d'une livre """
    browser.get(url_d_un_livre)
    html_page_listing = browser.page_source # permet de prendre le code html de la page 
    soup = BeautifulSoup(html_page_listing , 'lxml') # créer la soupe avec le parser
    alerte = soup.find_all("p", class_="a-last")   # pour faire le prog robuste je copie cette partie
    while  str(alerte) == """[<p class="a-last">Sorry, we just need to make sure you\'re not a robot. For best results, please make sure your browser is accepting cookies.</p>]""" :
        passer_le_captcha()
        browser.get(url_d_un_livre)
        html_page_listing = browser.page_source # permet de prendre le code html de la page
        soup = BeautifulSoup(html_page_listing , 'lxml')
        alerte = soup.find_all("p", class_="a-last") # à la fin de while 
    if soup.find_all( {'span'  : 'a-list-item'}) is not None : 
        rang_livre = soup.find_all( {'span'  : 'a-list-item'})
        if len(str(re.compile(regex_expression).search(str(rang_livre))).split())>5 : 
            rang = int(str(re.compile(regex_expression).search(str(rang_livre))).split()[5].replace("#","").replace(",",""))
        else : rang = None
    else : rang = None
    return(rang)


def titre_livre(url_d_un_livre):
    """fonction qui retourne le titre du livre lorsqu'on visite la page du livre """
    browser.get(url_d_un_livre)
    html_page_listing = browser.page_source # permet de prendre le code html de la page 
    soup = BeautifulSoup(html_page_listing , 'lxml') # créer la soupe avec le parser
    #     if soup.find('span',class_ = "a-size-extra-large celwidget").text == None: 
    #         pass
    #     else: 
    alerte = soup.find_all("p", class_="a-last")   # pour faire le prog robuste je copie cette partie
    while  str(alerte) == """[<p class="a-last">Sorry, we just need to make sure you\'re not a robot. For best results, please make sure your browser is accepting cookies.</p>]""" :
        passer_le_captcha()
        browser.get(url_d_un_livre)
        html_page_listing = browser.page_source # permet de prendre le code html de la page
        soup = BeautifulSoup(html_page_listing , 'lxml')
        alerte = soup.find_all("p", class_="a-last") # à la fin de while 
    titre=soup.find('span',class_ = "a-size-extra-large celwidget").text.strip()
    return(titre)


def lien_livre(url_d_un_livre):
    """fonction qui retour le lien du livre  """
    return(url_d_un_livre)
    


In [64]:
y_a_t_il_des_livres_dans_page(urlcaptcha)

MaxRetryError: HTTPConnectionPool(host='localhost', port=50523): Max retries exceeded with url: /session/97ac7bc4e5202e642b54fd73afc62f43/url (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0xa59ed8e0>: Failed to establish a new connection: [Errno 111] Connection refused'))

In [65]:
def scrapper_un_livre(url_d_un_livre):
    
    browser.get(url_d_un_livre)
    html_page_listing = browser.page_source # permet de prendre le code html de la page 
    soup = BeautifulSoup(html_page_listing , 'lxml') # créer la soupe avec le parser
    alerte = soup.find_all("p", class_="a-last")   # pour faire le prog robuste je copie cette partie
    while  str(alerte) == """[<p class="a-last">Sorry, we just need to make sure you\'re not a robot. For best results, please make sure your browser is accepting cookies.</p>]""" :
        passer_le_captcha()
        browser.get(url_d_un_livre)
        html_page_listing = browser.page_source # permet de prendre le code html de la page
        soup = BeautifulSoup(html_page_listing , 'lxml')
        alerte = soup.find_all("p", class_="a-last") # à la fin de while 
    titre=soup.find('span',class_ = "a-size-extra-large celwidget").text.strip() #retour le titre du livre
    
    if soup.find_all( {'span'  : 'a-list-item'}) is not None : 
        rang_livre = soup.find_all( {'span'  : 'a-list-item'})
        if len(str(re.compile(regex_expression).search(str(rang_livre))).split())>5 : 
            rang = int(str(re.compile(regex_expression).search(str(rang_livre))).split()[5].replace("#","").replace(",",""))
        else : rang = None
    else : rang = None
    
    return(list((titre,rang,url_d_un_livre)))

    

In [ ]:
scrapper_un_livre("https://www.amazon.com/Brunelleschis-Dome-Renaissance-Reinvented-Architecture-ebook/dp/B00DUVKOUW/ref=sr_1_1?qid=1707170304&s=digital-text&sr=1-1")

In [ ]:
tableau_principal = []
for i in range(0,2): # i = 0 #doit aller de 0 à 5136 il y a 5137 éléments categoire
    ligne_categorie = liste_categorie.iloc[i].tolist()
    #prends un éléments du dataframe en liste
    # permet de convertir un élément du dataframe en liste
    for j in range(1,3):# doit aller de 1 à ce qu'il n'y ait plus de page mais on peut partir de 1 à 2 ou 3 au départ.
        page_livres_multi=trouver_node_url_a_incrementer(liste_categorie.iloc[i].tolist()[0]) + str(j)
        page_livres_multi
        print(page_livres_multi)

In [ ]:
ligne_categorie = liste_categorie.iloc[1].tolist()
ligne_categorie[0]

In [28]:
page_livres_multi = trouver_node_url_a_incrementer(ligne_categorie[0])+str(1)
page_livres_multi

'https://www.amazon.com/s?i=digital-text&rh=n%urlcaptcha&fs=true&page=1'

In [29]:
trouver_les_livres_dans_page(page_livres_multi)

['https://www.amazon.com/Women-Novel-Kristin-Hannah-ebook/dp/B0C1X97LW7/ref=sr_1_1?dib=eyJ2IjoiMSJ9.yWEagzc0RvQ58fdJ4b61BEaVYdYGlOcrNJ1TZNCSCQyZcUnKaUb7isYZbB9UxTdAlk8RzbfVt-LP1fby8D7pwhyrDShp06VJJKjuMzZl_JtiRBvWRI1-KBWtZ52_t4kFYt7hv2Ne6rVbFWHNUVoqXM3q7xq3sGRaq7ON-8Xp8pyWEOFB5nTPmOwUj560aZ2ZEowZUH9uzDyrL_hA_pRp7fJAJTLd1BiwZnBbAm5jsPM.Cg-lsjzdEKO5kY21OKfiVtYZyWcKid8xoA1jNIDo7AA&dib_tag=se&qid=1709589997&s=digital-text&sr=1-1',
 'https://www.amazon.com/Iron-Flame-Empyrean-Book-2-ebook/dp/B0C4JMJBNX/ref=sr_1_2?dib=eyJ2IjoiMSJ9.yWEagzc0RvQ58fdJ4b61BEaVYdYGlOcrNJ1TZNCSCQyZcUnKaUb7isYZbB9UxTdAlk8RzbfVt-LP1fby8D7pwhyrDShp06VJJKjuMzZl_JtiRBvWRI1-KBWtZ52_t4kFYt7hv2Ne6rVbFWHNUVoqXM3q7xq3sGRaq7ON-8Xp8pyWEOFB5nTPmOwUj560aZ2ZEowZUH9uzDyrL_hA_pRp7fJAJTLd1BiwZnBbAm5jsPM.Cg-lsjzdEKO5kY21OKfiVtYZyWcKid8xoA1jNIDo7AA&dib_tag=se&qid=1709589997&s=digital-text&sr=1-2',
 'https://www.amazon.com/House-Flame-Shadow-Crescent-City-ebook/dp/B0BY9M2C4X/ref=sr_1_3?dib=eyJ2IjoiMSJ9.yWEagzc0RvQ58fdJ4b61BEaVYdYGlOcrNJ

In [30]:
url="https://www.amazon.com/s?i=digital-text&rh=n%3A157631011&fs=true&page=2"

In [43]:
browser.get(url)

NameError: name 'url' is not defined

In [32]:
trouver_les_livres_dans_page("https://www.amazon.com/s?i=digital-text&rh=n%3A157634011&fs=true&qid=1707170086&ref=sr_pg_1")

['https://www.amazon.com/Brunelleschis-Dome-Renaissance-Reinvented-Architecture-ebook/dp/B00DUVKOUW/ref=sr_1_1?dib=eyJ2IjoiMSJ9.nV8HB4qbL2lA2XI4csb__bc8aR2ZFHQR-RzKtMoIPph_Axwg7L3KbbfS21SsLjQtF2iWLFfjv6DWJrvdU7WDJmOPUMZB97a15bPAqQ6PynXvo25PWm09k2lGt1o36c6rINDR5NVQvF29CXwfa_OHWzDVhBmBpGbv2CzSEUAjFlR9J-LwXAY_MT-dN3iFW2SAfa25qHHWeJyIyVc4QAVhPNOufAkYRKWswSt8philfkA.HngmPBDYBTEKoJmy_opT4iFjTX7pp2qyR0QoJ74IE4s&dib_tag=se&qid=1709590008&s=digital-text&sr=1-1',
 'https://www.amazon.com/Universe-Stone-Chartres-Cathedral-Invention-ebook/dp/B001ANYCW8/ref=sr_1_2?dib=eyJ2IjoiMSJ9.nV8HB4qbL2lA2XI4csb__bc8aR2ZFHQR-RzKtMoIPph_Axwg7L3KbbfS21SsLjQtF2iWLFfjv6DWJrvdU7WDJmOPUMZB97a15bPAqQ6PynXvo25PWm09k2lGt1o36c6rINDR5NVQvF29CXwfa_OHWzDVhBmBpGbv2CzSEUAjFlR9J-LwXAY_MT-dN3iFW2SAfa25qHHWeJyIyVc4QAVhPNOufAkYRKWswSt8philfkA.HngmPBDYBTEKoJmy_opT4iFjTX7pp2qyR0QoJ74IE4s&dib_tag=se&qid=1709590008&s=digital-text&sr=1-2',
 'https://www.amazon.com/Roman-Pilgrimage-Churches-George-Weigel-ebook/dp/B00BVTSD0S/ref=sr_1_3

In [66]:
browser = webdriver.Chrome(executable_path=DRIVER_PATH_chromedriver)
browser.get("https://www.amazon.com/errors/validateCaptcha")
browser.get(urlcaptcha)
html_page_listing = browser.page_source # permet de prendre le code html de la page 
soup = BeautifulSoup(html_page_listing , 'lxml') # créer la soupe avec le parser
alerte = soup.find_all("p", class_="a-last")
if str(alerte) == """[<p class="a-last">Sorry, we just need to make sure you\'re not a robot. For best results, please make sure your browser is accepting cookies.</p>]""" :
    passer_le_captcha()
    browser.implicitly_wait(10)
    browser.refresh()
    soup = BeautifulSoup(html_page_listing , 'lxml') # créer la soupe avec le parser
else : 
    print(soup)
# title = movie.find("p").find("a-last").string

/tmp/ipykernel_2374/3735585875.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  browser = webdriver.Chrome(executable_path=DRIVER_PATH_chromedriver)


In [34]:
soup

<html class="a-no-js" lang="en-us"><!--<![endif]--><head>
<meta content="text/html; charset=utf-8" http-equiv="content-type"/>
<meta charset="utf-8"/>
<meta content="IE=edge,chrome=1" http-equiv="X-UA-Compatible"/>
<title dir="ltr">Amazon.com</title>
<meta content="width=device-width" name="viewport"/>
<link href="https://images-na.ssl-images-amazon.com/images/G/01/AUIClients/AmazonUI-3c913031596ca78a3768f4e934b1cc02ce238101.secure.min._V1_.css" rel="stylesheet"/>
<script>

if (true === true) {
    var ue_t0 = (+ new Date()),
        ue_csm = window,
        ue = { t0: ue_t0, d: function() { return (+new Date() - ue_t0); } },
        ue_furl = "fls-na.amazon.com",
        ue_mid = "ATVPDKIKX0DER",
        ue_sid = (document.cookie.match(/session-id=([0-9-]+)/) || [])[1],
        ue_sn = "opfcaptcha.amazon.com",
        ue_id = 'GVT2Q9BCKT5RRZVYB26H';
}
</script>
<script src="https://images-na.ssl-images-amazon.com/images/G/01/csminstrumentation/csm-captcha-instrumentation.min.js"></scr

In [61]:
browser.get(url_page_principale)
    html_page_listing = browser.page_source # permet de prendre le code html de la page 
    soup = BeautifulSoup(html_page_listing , 'lxml') # créer la soupe avec le parser
    livres = soup.find_all('a',class_ = 'a-link-normal s-underline-text s-underline-link-text s-link-style a-text-normal')
    liens_livres=['https://www.amazon.com'+livre.get('href') for livre in livres]  
    #on utilise liste comprehension pour prendre l'élement href (lien hypertexte) 
    # et ajouter www.amazon.com/ pour avoir un url exploitable
    return(liens_livres)

IndentationError: unexpected indent (2739723312.py, line 2)

In [67]:
tableau_principal = []
start = time.time()


for i in range(1,5136): # i = 0 #doit aller de 0 à 5136 il y a 5137 éléments categoire
    ligne_categorie = liste_categorie.iloc[i].tolist()
    #prends un éléments du dataframe en liste
    # permet de convertir un élément du dataframe en liste
    for j in range(1,2):# doit aller de 1 à ce qu'il n'y ait plus de page mais on peut partir de 1 à 2 ou 3 au départ.
        page_livres_multi=trouver_node_url_a_incrementer(liste_categorie.iloc[i].tolist()[0]) + str(j)
        page_livres_multi
    # for livre in page_livres_multi:
    # trouver_les_livres_dans_page(page_livres_multi)  # faire un joli code utiliser que la fonction retourne une liste et balayer la liste
    
    # for k in range(1,10):
    #     trouver_les_livres_dans_page(page_livres_multi)[k]
    #     page_livre = trouver_les_livres_dans_page(page_livres_multi)[k]
    #     liste_livre = [titre_livre(page_livre) , rang_livre(page_livre), lien_livre(page_livre)]
    #     ligne_supp = ligne_categorie + liste_livre
    #     tableau_principal.append(ligne_supp)

        livres_a_scrapper = trouver_les_livres_dans_page(page_livres_multi)  # faire un joli code utiliser que la fonction retourne une liste et balayer la liste

        for livre in livres_a_scrapper:
            ligne_supp = ligne_categorie + scrapper_un_livre(livre)
            tableau_principal.append(ligne_supp)
end = time.time()
print(end - start)
tableau_principal

NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=109.0.5414.112)
Stacktrace:
#0 0x000000a98e58 <unknown>
#1 0x00000083f264 <unknown>
#2 0x000000815870 <unknown>
#3 0x0000008aa1b8 <unknown>
#4 0x0000008bec28 <unknown>
#5 0x0000008a5018 <unknown>
#6 0x000000872380 <unknown>
#7 0x000000873668 <unknown>
#8 0x000000ae7818 <unknown>
#9 0x000000aea6e8 <unknown>
#10 0x000000ae9f8c <unknown>
#11 0x000000aeaea8 <unknown>
#12 0x000000ad08e8 <unknown>
#13 0x000000aeb320 <unknown>
#14 0x000000ac2a50 <unknown>
#15 0x000000b0b2f8 <unknown>
#16 0x000000b0b47c <unknown>
#17 0x000000b272f8 <unknown>
#18 0x0000b6e90300 start_thread
#19 0x000000000000 <unknown>


In [61]:
titres_colonnes = ["url_categorie","cat1","cat2","cat3","cat4","cat5","cat6","titre","rang","lien"]

In [62]:
df = pd.DataFrame(tableau_principal, columns=titres_colonnes)

In [63]:
df

,url_categorie,cat1,cat2,cat3,cat4,cat5,cat6,titre,rang,lien
0,https://www.amazon.com/s?bbn=157630011&rh=n%3A...,Arts & Photography,Architecture,"Architects, A-Z",NaN,NaN,NaN,A Place of My Own: The Architecture of Daydreams,207610.0,https://www.amazon.com/Place-My-Own-Architectu...
1,https://www.amazon.com/s?bbn=157630011&rh=n%3A...,Arts & Photography,Architecture,"Architects, A-Z",NaN,NaN,NaN,Architectural Digest at 100: A Century of Style,50708.0,https://www.amazon.com/Architectural-Digest-10...
2,https://www.amazon.com/s?bbn=157630011&rh=n%3A...,Arts & Photography,Architecture,"Architects, A-Z",NaN,NaN,NaN,The Stoic way of Life: The Ultimate Guide of S...,81682.0,https://www.amazon.com/Stoic-way-Life-Confiden...
3,https://www.amazon.com/s?bbn=157630011&rh=n%3A...,Arts & Photography,Architecture,"Architects, A-Z",NaN,NaN,NaN,The $12 Million Stuffed Shark: The Curious Eco...,614493.0,https://www.amazon.com/Million-Stuffed-Shark-E...
4,https://www.amazon.com/s?bbn=157630011&rh=n%3A...,Arts & Photography,Architecture,"Architects, A-Z",NaN,NaN,NaN,The Gardner Heist: The True Story of the World...,256404.0,https://www.amazon.com/Gardner-Heist-Worlds-La...
...,...,...,...,...,...,...,...,...,...,...
155,https://www.amazon.com/s?bbn=154781011&rh=n%3A...,Biographies & Memoirs,Historical,Africa,NaN,NaN,NaN,Empress of the East: How a European Slave Girl...,NaN,https://www.amazon.com/Empress-East-European-B...
156,https://www.amazon.com/s?bbn=154781011&rh=n%3A...,Biographies & Memoirs,Historical,Africa,NaN,NaN,NaN,Blood Brothers: The Dramatic Story of a Palest...,NaN,https://www.amazon.com/Blood-Brothers-Dramatic...
157,https://www.amazon.com/s?bbn=154781011&rh=n%3A...,Biographies & Memoirs,Historical,Africa,NaN,NaN,NaN,Don't Let's Go to the Dogs Tonight: An African...,NaN,https://www.amazon.com/Dont-Lets-Dogs-Tonight-...
158,https://www.amazon.com/s?bbn=154781011&rh=n%3A...,Biographies & Memoirs,Historical,Africa,NaN,NaN,NaN,The Last Mughal,NaN,https://www.amazon.com/Last-Mughal-William-Dal...


In [76]:
df.to_csv('2024-0.csv')